### Grupos

1. 
2. 
3.
4.
5.
6.
7.

### Atividades

1. Ler as bases de preços diários
2. Calcular retornos relativos diários (HP=1)
3. Calcular retorno médio e desvio padrão de cada ativo
4. Calcular matriz de covariância dos ativos
5. Plotar os ativos num gráfico de dispersão: retorno x risco
6. Definir pesos para os ativos e compor em uma carteira. Calcular o retorno e o risco desta carteira
7. Simular diversos conjuntos de pesos aleatoriamente e plotar no gráfico de dispersão estas carteiras

### Dicas

1. Defina funções para ler as bases ao invés de repetir os códigos.
2. Utilize as funções do Pandas - .mean(), .cov(), .shift(), etc.
3. Verifique os resultados a cada cálculo.
4. Para multiplicação matricial, utilize: np.dot()
5. Para simular valores aleatórios, utilize: np.ramdon.ramdon()

In [ ]:
# Bibliotecas
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
import scipy.optimize as solver

In [ ]:
# Ler e tratar as bases
def read_infoMoney(file_name):
    
    return df

In [ ]:
# Dicionário de ativos
assets = {'IBOV':  'Ibovespa (IBOV) - Histórico  InfoMoney.csv',
          'PETR4': 'Petrobras PETR4 - Histórico  InfoMoney.csv',
          'MRFG3': 'Marfrig MRFG3 - Histórico  InfoMoney.csv',
          'MGLU3': 'Magazine Luiza MGLU3 - Histórico  InfoMoney.csv',
          'BBAS3': 'Banco do Brasil BBAS3 - Histórico  InfoMoney.csv',
          'BBDC3': 'Bradesco BBDC3 - Histórico  InfoMoney.csv',
          'ITUB3': 'Itaú Unibanco ITUB3 - Histórico  InfoMoney.csv',
          'VALE3': 'Vale S.A. VALE3 - Histórico  InfoMoney.csv'}
assets

In [ ]:
# Consolidando os dados em um único DF
df_assets = pd.DataFrame()
 

In [ ]:
# Calcular retornos
def calc_return(df, hp):
    return df_returns


In [ ]:
# Cálculo da correlação entre os ativos
correl_matrix =

cov_matrix = 

In [ ]:
# Cálculo do retorno e desvio padrão dos ativos
mi =  # média
sigma =  # desvio padrão



In [ ]:
# Gráfico dos ativos: retorno x risco


In [ ]:
# Compondo uma carteira 


In [ ]:
# Cálculo de retorno e risco da carteira
mi_P = 
sigma_P = 

print(f'Retorno esperado: {round(mi_P*100, 4)}%, risco: {round(sigma_P*100, 4)}%')

In [ ]:
# Simulando carteiras com pesos diferentes

    
# Gráfico
   

## Otimização de portfólio

Todo processo de otimização necessita da elaboração de uma função objetiva e da definição de um conjunto de restrições (se houver). Por exemplo, podemos minimizar uma função custo ou maximizar uma função de lucro. No nosso caso, utilizando a teoria de portfólio de Markowitz, desejamos uma das duas coisas:
- minimizar a volatilidade da carteira ou
- maximizar os retornos esperados

Analisaremos adiante cada um dos casos.

### Minimizar a volatilidade

Neste caso, desejamos obter um conjunto de pesos para os ativos da análise, tais que o risco da carteira ponderada por estes pesos seja o menor possível. Podemos escrever este objetivo da seguinte forma:

$$ min F(w) = \sqrt{\sum_{i=1}^N \sum_{j=1}^N W_i \cdot W_j \cdot COV_{i,j}} $$

Contudo, devemos considerar nossas restrições. A primeira delas refere-se aos valores dos pesos. Por definição, estes devem sempre somar 1. Portanto:

$$\sum_{i=1}^N W_i = 1 $$

ou

$$\sum_{i=1}^N W_i - 1 = 0 $$

A segunda restrição refere-se ao retorno desejado para a carteira, $\mu$. Para que este objetivo seja alcançado, podemos definir:

$$\sum_{i=1}^N W_i r_i = \mu $$

ou

$$\sum_{i=1}^N W_i r_i - \mu = 0 $$

Por fim, precisamos que os pesos possuam valores positivos e sejam menores que 100\%:

$$ w_i >= 0, \forall i=1,...,n $$
$$ w_i <= 1, \forall i=1,...,n $$

O que o cálculo numérico realiza é uma busca para encontrar o melhor conjunto de pesos para cada ativo que garanta que as restrições são respeitadas e que se tenha o menor risco possível.

---

Biblioteca Python utilizada para otimização:

https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html

---


In [ ]:
# Definindo função objetivo
def f_obj_min_risk(w):
    return np.sqrt(np.dot(w, np.dot(cov_matrix, w)))

# Definindo valor inicial
w_ini = np.array([1/len(mi)]* len(mi)) 

In [ ]:
# Definindo o retorno desejado
ret = 0.004

# Definindo restrições
constraints = [{'type': 'eq', 'fun': lambda x: sum(x) - 1}, # primeira restrição
               {'type': 'eq', 'fun': lambda x: sum(x*mi) - ret}] # segunda restrição

# Definindo limites para os pesos
bounds = tuple((0, 1) for x in range(len(w))) 

In [ ]:
result = solver.minimize(f_obj_min_risk, w_ini, constraints=constraints, bounds=bounds, 
                         method='SLSQP')
result

In [ ]:
pesos = [round(w*100, 2) for w in result.x]
print(f'Pesos (%): {pesos}\nRisco portfólio (%): {round(result.fun*100,2)}')

In [ ]:
# Simulando para diversos retornos (buscando a fronteira eficiente)
ret_range = np.arange(0.0, 0.008, 0.0001)
sigma_sim = []

for ret in ret_range:
    constraints = [{'type': 'eq', 'fun': lambda x: sum(x) - 1}, # primeira restrição
                   {'type': 'eq', 'fun': lambda x: sum(x*mi) - ret}] # segunda restrição
    
    result = solver.minimize(f_obj_min_risk, w_ini, constraints=constraints, bounds=bounds, 
                         method='SLSQP')
    sigma_sim.append(result.fun)
    
plt.plot(sigma_sim, ret_range, '--')

### Maximizar o retorno
Neste caso, fixamos a volatilidade desejada $\sigma_P$ e definimos a função objetivo para ampliar os retornos calculados a partir dos pesos. O problema de otimização pode ser rezumido por:

$$ max F(w) = \sum_{i=1}^N W_i r_i $$

Sujeito a:

$$\sqrt{\sum_{i=1}^N \sum_{j=1}^N W_i \cdot W_j \cdot COV_{i,j}} = \sigma_P $$
$$\sum_{i=1}^N W_i - 1 = 0 $$
$$ w_i >= 0, \forall i=1,...,n $$
$$ w_i <= 1, \forall i=1,...,n $$

Nas bibliotecas existentes, os problemas de otimização sempre são descritas como minimização. Mas isto não é um problema, pois podemos simplesmente inverter o sinal da função objetivo:

$$ min F(w) = - \sum_{i=1}^N W_i r_i $$

In [ ]:
# Definindo função objetivo

# Definindo valor inicial

# Definindo o retorno desejado

# Definindo restrições

# Definindo limites para os pesos


In [ ]:
# Imprime resultado


In [ ]:
# Simulando para diversos riscos (buscando a fronteira eficiente)


In [ ]:
# Gráfico 
